In [0]:
spark

In [0]:
orders_schema = "order_id long, order_date string, customer_id long, order_status string"

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/Datasets/orders")

True

In [0]:
from pyspark.sql.functions import *

In [0]:
order_df=spark.readStream.format("csv").schema(orders_schema).option("path","dbfs:/FileStore/Datasets/orders").load()

In [0]:
order_df.createOrReplaceTempView("orders")

In [0]:
explode_df=spark.sql("select customer_id,order_status,count(order_status) as count from orders group by customer_id,order_status")

In [0]:
spark.sql("create database if not exists retaildb")

DataFrame[]

In [0]:
streaming_query = explode_df \
.writeStream \
.format("delta") \
.outputMode("complete") \
.option("checkpointLocation","checkpointdir6") \
.toTable("retaildb.orders_data")

In [0]:
spark.sql("select * from retaildb.orders_data").show()

+-----------+---------------+-----+
|customer_id|   order_status|count|
+-----------+---------------+-----+
|      12111|       COMPLETE|    1|
|       7776|PENDING_PAYMENT|    1|
|       5657|PENDING_PAYMENT|    1|
|       5649|        PENDING|    1|
|      12092|PENDING_PAYMENT|    1|
|       9776|        PENDING|    1|
|      10039|PENDING_PAYMENT|    1|
|        256|PENDING_PAYMENT|    1|
|       4367|PENDING_PAYMENT|    2|
|       5648|PENDING_PAYMENT|    1|
|      12186|     PROCESSING|    1|
|       9198|     PROCESSING|    2|
|       3960|       COMPLETE|    1|
|       4840|       COMPLETE|    1|
|       7130|       COMPLETE|    1|
|       1549|        ON_HOLD|    1|
|       2711|        PENDING|    2|
|      11599|         CLOSED|    1|
|       9503|         CLOSED|    2|
|       5863|         CLOSED|    1|
+-----------+---------------+-----+
only showing top 20 rows



In [0]:
streaming_query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/Datasets/completemode/")

True